# <center>Data Analysis 3 - Assignment 2<center>
    
<center>Created by Zsófia Rebeka Katona</center>

Methods to use:
- ols
- tree based method (bagging or boosting)
- can be lasso
- RF or RGB
- the variables may be important
- you can put all the things
- the model selects it itself
- be able to interpret what I'm doing
- even with lasso, I can't select all the variables, all the possible interactions
- first do a RF and based on the VI, the top 10 most important variables -> build a LASSO model
- we have to explain what drives what
- you pick the most important variables
- it gives us e birdseye view on the relationship between the x and y variables
- 

In [1]:
# Importing the required libraries
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import os
from pathlib import Path
import sys
from patsy import dmatrices
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.impute import SimpleImputer
from sklearn.inspection import permutation_importance
from sklearn.inspection import PartialDependenceDisplay
from sklearn.inspection import partial_dependence
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso
import patsy
from sklearn.model_selection import RepeatedKFold
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
import statsmodels.formula.api as smf
from stargazer import stargazer
from statsmodels.tools.eval_measures import mse,rmse
import math

import ast
import seaborn as sns
from plotnine import *

In [2]:
# Importing the prewritten helper functions
from py_helper_functions import *

____________________________________________________________________________________________________________________________
# Data importing

In [4]:
# Importing data from github
data = pd.read_csv('https://raw.githubusercontent.com/zsofiarebeka/DA3_Assignment-2/main/Toronto_airbnb_listings.csv')

In [5]:
# Checking the data
data.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,1419,Home in Toronto · ★5.0 · 5 bedrooms · 7 beds ·...,1565,Alexandra,NaN,Little Portugal,43.64590,-79.42423,Entire home/apt,NaN,28,6,2017-08-07,0.06,1,0,0,NaN
1,8077,Rental unit in Toronto · ★4.84 · 1 bedroom · 1...,22795,Kathie & Larry,NaN,Waterfront Communities-The Island,43.64080,-79.37673,Private room,100.0,180,169,2013-08-27,0.96,2,0,0,NaN
2,26654,Condo in Toronto · ★4.79 · 1 bedroom · 2 beds ...,113345,Adela,NaN,Waterfront Communities-The Island,43.64608,-79.39032,Entire home/apt,145.0,28,42,2023-09-01,0.27,5,77,2,NaN
3,27423,Rental unit in Toronto · ★4.93 · Studio · 1 be...,118124,Brent,NaN,South Riverdale,43.66884,-79.32725,Entire home/apt,75.0,90,28,2023-08-31,0.17,1,215,1,NaN
4,30931,Rental unit in Toronto · 1 bedroom · 2 beds · ...,22795,Kathie & Larry,NaN,Waterfront Communities-The Island,43.64015,-79.37625,Entire home/apt,134.0,180,1,2010-08-11,0.01,2,0,0,NaN


In [6]:
# Checking out the info
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20386 entries, 0 to 20385
Data columns (total 18 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              20386 non-null  int64  
 1   name                            20386 non-null  object 
 2   host_id                         20386 non-null  int64  
 3   host_name                       20384 non-null  object 
 4   neighbourhood_group             0 non-null      float64
 5   neighbourhood                   20386 non-null  object 
 6   latitude                        20386 non-null  float64
 7   longitude                       20386 non-null  float64
 8   room_type                       20386 non-null  object 
 9   price                           16595 non-null  float64
 10  minimum_nights                  20386 non-null  int64  
 11  number_of_reviews               20386 non-null  int64  
 12  last_review                     

__________________________________________________________________________________________________________________________
## Data cleaning

In [7]:
# Dropping columns that are not needed
data = data.drop(columns = ["listing_url",
                            "scrape_id",
                            "last_scraped",
                            "name",
                            "description",
                            "neighborhood_overview",
                            "picture_url",
                            "neighbourhood_group_cleansed",
                            "bathrooms",
                            "host_url",
                            "host_location",
                            "host_about",
                            "host_response_time",
                            "host_thumbnail_url",
                            "host_picture_url",
                            "license"
                            ])

KeyError: "['listing_url', 'scrape_id', 'last_scraped', 'description', 'neighborhood_overview', 'picture_url', 'neighbourhood_group_cleansed', 'bathrooms', 'host_url', 'host_location', 'host_about', 'host_response_time', 'host_thumbnail_url', 'host_picture_url'] not found in axis"

__________________________________________________________________________________________________________________________
## Feature Engineering

In [ ]:
# Turning the columns with t/f values to binary columns and rename them
binary_columns = ["host_is_superhost",
                  "host_has_profile_pic",
                  "host_identity_verified",
                  "has_availability",
                  "instant_bookable"
                 ]
for col in binary_columns:
    data["d_" + col] = data[col].map({"t": 1, "f": 0})

In [ ]:
data.T

In [ ]:
# Formatting columns with percentages
for perc in ["host_response_rate", "host_acceptance_rate"]:
    data[perc] = data[perc].replace("%", "", regex=True)
    data[perc] = pd.to_numeric(data[perc], errors="coerce")

In [ ]:
# Drop the untransformed binary columns
for col in binary_columns:
    data.drop(columns=[col], inplace=True)

In [ ]:
# Coverting time-related columns to datetime type of objects
datetime_columns = ["host_since",
                    "calendar_updated",
                    "calendar_last_scraped",
                    "first_review",
                    "last_review"
                    ]
for col in datetime_columns:
    data[col] = pd.to_datetime(data[col], errors = 'coerce') 

In [ ]:
# Cleaning the price column from currency symbols and decimals
data.price = data.price.str.replace("$", "").str.replace(",", "")

# Removing rows with empty strings to be able to convert to float
data = data[data["price"] != ""]

# Turning the price column to float
data["price"] = data["price"].astype(float)

# Rename the price columns
data.rename(columns={"price": "usd_price_day"}, inplace=True)

# Dropping rows where price is missing
data = data.dropna(subset=['usd_price_day'])

In [ ]:
# Filtering for small and mid-size apartments hosting 2-6 guests
data = data[(data.accommodates >= 2) & (data.accommodates <= 6)]

#### Property types variable

In [ ]:
# Checking the property types and their number
data.property_type.value_counts()

In [ ]:
# Only keeping the property types if they are small and mid-size apartments
data = data.loc[lambda x: x["property_type"].isin(["Entire rental unit",
                                                   "Entire condo",
                                                   "Private room in home",
                                                   "Entire home",
                                                   "Private room in rental unit",
                                                   "Entire loft",
                                                   "Entire serviced apartment",
                                                   "Entire home/apt",
                                                   "Shared room in rental unit",
                                                   "Shared room in home"
                                                  ])]

In [ ]:
# Categorizing the properties to Rental Unit or House catgeories
data['property_type'] = data['property_type'].apply(lambda x: 'Rental Unit' if "rental unit" in x else 'House')

#### Room types and bed variables

In [ ]:
# Checking the room types
data.value_counts("room_type")

In [ ]:
# Renaming the room types as they are too long
data["room_type"] = data["room_type"].map({"Entire home/apt": "Entire/Apt",
                                          "Private room": "Private",
                                          "Shared room" : 'Shared'})

In [ ]:
# Checking the bed types
data.beds.value_counts()

#### Bathrooms variable

In [ ]:
# Checking the bathroom types
data["bathrooms_text"].value_counts()

In [ ]:
# Assigning numerical values to bathrooms
data["bathrooms_text"] = data["bathrooms_text"].map({"1 bath": 1,
                                                  "2 baths": 2,
                                                  "1 private bath": 1,
                                                  "1 shared bath": 1,
                                                  "1.5 baths": 1.5,
                                                  "2.5 baths": 2.5,
                                                  "1.5 shared baths": 1.5,
                                                  "2 shared baths": 2,
                                                  "3 baths": 3,
                                                  "3.5 baths": 3.5,
                                                  "4 baths": 4,
                                                  "3 shared baths": 3,
                                                  "0 baths": 0,
                                                  "5 baths": 5,
                                                  "4.5 baths" : 4.5,
                                                  "5.5 baths" : 5.5,
                                                  "8 baths" : 8,
                                                  "Private half-bath" : 0.5
                                                  })

In [ ]:
# Renaming the bathrooms_text column to bathrooms
data.rename(columns={"bathrooms_text": "bathrooms"}, inplace = True)

### Pooling values

In [ ]:
# Pooling accomodations with 0, 1, 2-10 bathrooms

bins = pd.IntervalIndex.from_tuples([(0, 1), (1, 2), (2, 10)], closed="left")
f_bath = pd.cut(data["bathrooms"].to_list(), bins, labels=["0.0", "1.0", "2.0"])
f_bath.categories = [0, 1, 2]
data["bathrooms"] = f_bath

f_bath.value_counts(dropna=False)

In [ ]:
data.number_of_reviews.describe()

In [ ]:
# Pooling the number of reviews into three categories: none, 1-27 and over 27

bins = pd.IntervalIndex.from_tuples(
    [(0, 1), (1, 27), (27, max(data.number_of_reviews))], closed="left"
)
fnor = pd.cut(data["number_of_reviews"].to_list(), bins, labels=["0", "1", "2"])
fnor.categories = [0, 1, 2]
data["number_of_reviews"] = fnor
data["number_of_reviews"].value_counts(dropna=False)

In [ ]:
# Pooling and categorizing the number of minimum nights: 1,2,3 and 3+

bins = pd.IntervalIndex.from_tuples(
    [(1, 2), (2, 3), (3, max(data.minimum_nights))], closed="left"
)
f_min_n = pd.cut(data["minimum_nights"].to_list(), bins, labels=["1", "2", "3"])
f_min_n.categories = [1, 2, 3]
data["minimum_nights"] = f_min_n
data["minimum_nights"].value_counts(dropna=False)

### Working with missing values

In [ ]:
# Transforming the Infinite values with NaNs
data = data.replace([np.inf, -np.inf], np.nan)

In [ ]:
# Checking for missing values
to_filter = data.isna().sum()
to_filter[to_filter > 0]

In [ ]:
# In the case of target variable, we have to drop them
data = data.loc[lambda x: x.usd_price_day.notnull()]

# Making sure ther are no NA values in the target variable
data.usd_price_day.isna().value_counts()

In [ ]:
# When there is a few and not that important, we can impute them
data = data.assign(
    host_response_rate=lambda x: x["host_response_rate"].fillna(np.median(x["host_response_rate"].dropna())),
    host_acceptance_rate=lambda x: x["host_acceptance_rate"].fillna(np.median(x["host_acceptance_rate"].dropna())),
    bathrooms=lambda x: x["bathrooms"].fillna(np.median(x["bathrooms"].dropna())),
    beds=lambda x: np.where(x["beds"].isnull(), x["accommodates"], x["beds"]),
    minimum_nights=lambda x: x["minimum_nights"].fillna(1),
    number_of_reviews=lambda x: x["number_of_reviews"].fillna(1)
)

In [ ]:
# When there are too many missing values and the column is not that important, we can drop the column
data=data.drop(columns = ["bedrooms",
                "calendar_updated",
                "host_neighbourhood",
                "neighbourhood"])

In [ ]:
# Adding flags to the missing values where the variable is important

for var in ["first_review",
            "last_review",
            "review_scores_rating",
            "review_scores_accuracy",
            "review_scores_cleanliness",
            "review_scores_checkin",
            "review_scores_communication",
            "review_scores_location",
            "review_scores_value",
            "reviews_per_month"]:
    missing_values = data[var].isnull().sum()
    data['flag_' + var] = data[var].isnull().astype(int)
    print(f"Number of missing values in {var}: {missing_values}")

In [ ]:
# Checking if the flag columns are correct
data.flag_first_review.value_counts()

### Creating numerical values

In [ ]:
# Transforming and renaming the numerical columns

numericals = [
    "host_listings_count",
    "host_total_listings_count",
    "accommodates",
    "beds",
    "maximum_nights",
    "availability_30",
    "availability_60",
    "availability_90",
    "availability_365",
    "number_of_reviews_ltm",
    "review_scores_rating",
    "calculated_host_listings_count",
    "calculated_host_listings_count_entire_homes",
    "calculated_host_listings_count_private_rooms",
    "calculated_host_listings_count_shared_rooms",
    "reviews_per_month"
]

for col in numericals:
    data["n_" + col] = pd.to_numeric(data[col], errors="coerce")

In [ ]:
# Drop the untransformed binary columns
for col in numericals:
    data.drop(columns=[col], inplace=True)

### Creating factorial variables

In [ ]:
# Renaming columns to factorial

factorials = [
    "room_type",
    "neighbourhood_cleansed",
    "property_type",
    "number_of_reviews",
    "bathrooms",
    "minimum_nights"
]

for col in factorials:
    data.rename(columns={col: "f_" + col}, inplace=True)

__________________________________________________________________
# EDA
__________________________________________________________________

In [ ]:
# Checking out the total number of NA values left
data.isna().sum().sum()

#### Discovering the price variable

In [ ]:
# Checking the basic metrics for price
data.usd_price_day.describe(percentiles = [0.01, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]).map('{:,.1f}'.format)

In [ ]:
# Drop extreme values
data = data[data['usd_price_day'] <= 300]

In [ ]:
# Checking the distribution of airbnb prices
ggplot(data, aes(x='usd_price_day')) + \
    geom_histogram(fill='purple', color='purple', bins=100) + \
    theme_minimal() + \
    ggtitle('Price distribution of Airbnbs in Toronto') + \
    xlab('Price') + \
    ylab('Number of listings') 

In [ ]:
# Checking the distribution of hourly wages
ax = sns.histplot(data, x='usd_price_day', stat='density')
sns.kdeplot(data, x='usd_price_day', color="red")

ax.yaxis.set_visible(False)
plt.title('Distribution of prices (in USD)')
plt.xlabel('Price')
plt.ylabel('Number of listings');

In [ ]:
# Taking a look at the average price by property and room type
data.groupby(["f_property_type", "f_room_type"]).agg(mean_price=("usd_price_day", np.mean))

______________________________________________________________
# Modelling

____________________________________________________________________________


In [ ]:
# Creating a box plot groupped by property types
(
    ggplot(data, aes(x="f_property_type", y="usd_price_day"))
    + stat_boxplot(
        aes(group="f_property_type"),
        geom="errorbar",
        width=0.3,
        color=('#af8dc3', "#7fbf7b"),
        size=0.5,
        na_rm=True,
    )
    + geom_boxplot(
        aes(group="f_property_type"),
        color=('#af8dc3', "#7fbf7b"),
        size=0.5,
        width=0.6,
        alpha=0.3,
        na_rm=True,
        outlier_shape="",
    )
    + scale_y_continuous(expand=(0.01, 0.01), limits=(0, 300), breaks=seq(0, 301, 50))
    + labs(x="Property type", y="Price (US dollars)")
    + theme_bw()
)

In [ ]:
# Creating a box plot groupped by room types
(
    ggplot(data, aes(x="f_room_type", y="usd_price_day"))
    + stat_boxplot(
        aes(group="f_room_type"),
        geom="errorbar",
        width=0.3,
        color=(color[1], color[0], color[2]),
        size=0.5,
        na_rm=True,
    )
    + geom_boxplot(
        aes(group="f_room_type"),
        color=(color[1], color[0], color[2]),
        size=0.5,
        width=0.6,
        alpha=0.3,
        na_rm=True,
        outlier_shape="",
    )
    + scale_y_continuous(expand=(0.01, 0.01), limits=(0, 300), breaks=seq(0, 301, 50))
    + labs(x="Room type", y="Price (US dollars)")
    + theme_bw()
)

In [ ]:
# To see extreme values, it's better to plot the seaborn version as well
sns.boxplot(data = data, x = 'f_room_type', y = 'usd_price_day', orient = 'v')
plt.title('Price distribution of Airbnbs in Toronto by room type')
plt.xlabel('Room types')
plt.ylabel('Price per day (US dollars)')
plt.show()

In [ ]:
# We have to consider that there is a marginal value for shared room type
data.f_room_type.value_counts()

In [ ]:
# Looking at the prices based on the number of accommodates
(
    ggplot(
        data,
        aes(
            x="factor(n_accommodates)",
            y="usd_price_day",
            color="factor(f_property_type)",
        ),
        
    )
    + geom_boxplot(alpha=0.8, na_rm=True, outlier_shape="", width=0.8, stat="boxplot")
    + stat_boxplot(geom="errorbar", width=0.8, size=0.3, na_rm=True)
    + scale_color_manual(name="", values=(color[1], color[0]))
    + scale_fill_manual(name="", values=(color[1], color[0]))
    + labs(x="Accomodates (Persons)", y="Price (US dollars)",
          title = "Price distribution of houses and rental units by the number of accommodates")
    + scale_y_continuous(expand=(0.01, 0.01), limits=(0, 400), breaks=seq(0, 401, 50))
    + theme_bw()
    + theme(legend_position=(0.3, 0.8))
)

__________________________________________________________________
# 1. Model: OLS
__________________________________________________________________

#### Model 1. Regression of price on number of accommodates
$price=\beta_0 + \beta_1 accommodates$

In [ ]:
# Creating the regression for Model 1
reg1 = smf.ols(formula="usd_price_day~n_accommodates", data=data).fit(cov_type="HC1")

## Linear (OLS) vs  Non-linear (Lowess)

In [ ]:
# Plotting the OLS and Lowess models
sns.regplot(data, x = 'n_accommodates', y = 'usd_price_day', marker= '.', fit_reg= True, lowess= True, line_kws = {'color':'purple'})

# OLS
sns.regplot(data, x = 'n_accommodates', y = 'usd_price_day', scatter=False, line_kws={'color':'red', 'linestyle': '--'})

plt.title("Price per day compared to the number of accomodates in an apartment")
plt.xlabel('Number of accommodates in an apartment')
plt.ylabel('Price per day (in USD)')
plt.legend(['Observation', 'Lowess', 'OLS'])

plt.show()


The OLS and the Lowess plots are quite similar, therefore the linear model could easily find a good fit.

## Further models

#### Model 2. M1 + room types, property types, bathrooms, bedrooms, number of reviews

#### Model 3. M2 + neighbourhood cleansed

#### Model 4. M3 + several basic variables interactions

In [ ]:
# Creating the regressions
reg2 = smf.ols(formula="usd_price_day~n_accommodates + f_room_type + f_property_type + f_bathrooms + n_beds + f_number_of_reviews", data=data).fit(cov_type="HC1")
reg3 = smf.ols(formula="usd_price_day~n_accommodates + f_room_type + f_property_type + f_bathrooms + n_beds + f_number_of_reviews + f_neighbourhood_cleansed", data=data).fit(cov_type="HC1")
reg4 = smf.ols(formula="usd_price_day~n_accommodates + f_room_type + f_property_type + f_neighbourhood_cleansed + f_bathrooms + n_beds + f_number_of_reviews + f_room_type*f_property_type + n_accommodates*f_property_type + f_room_type*d_has_availability + f_bathrooms*f_property_type + f_neighbourhood_cleansed*f_property_type + d_instant_bookable*f_minimum_nights", data=data).fit(cov_type="HC1")

In [ ]:
# Viewing regression results with stargazer
sg = stargazer.Stargazer([reg1, reg2, reg3, reg4])
covariate_order = ["f_room_type[T.Private]", "f_room_type[T.Shared]", "f_property_type[T.Rental Unit]",
                   "n_accommodates", "f_bathrooms[T.1]", "f_bathrooms[T.2]", "n_beds",
                   "f_number_of_reviews[T.1]", "f_number_of_reviews[T.2]", "Intercept"]

# Set covariate order
sg.covariate_order(covariate_order)

# Rename covariates
sg.rename_covariates({"n_accommodates": "Accommodates", 
                      "f_room_type[T.Private]": "Private Room Type", 
                      "f_room_type[T.Shared]": "Shared Room Type", 
                      "f_property_type[T.Rental Unit]": "Rental Unit",
                      "f_bathrooms[T.1]" : "Bathrooms (1)",
                      "f_bathrooms[T.2]" : "Bathrooms (2-10)",
                      "n_beds" : "Beds",
                      "f_number_of_reviews[T.1]" : "Number of reviews (<27)",
                      "f_number_of_reviews[T.2]" : "Number of reviews (27<)",
                      "Intercept": "Constant"})

# Adding RMSE for measurement in the full sample for each model
rmse_values = [round(rmse(x.fittedvalues, data.usd_price_day), 3) for x in [reg1, reg2, reg3, reg4]]
sg.add_line('RMSE', rmse_values, location=stargazer.LineLocation.FOOTER_BOTTOM)

# Adding BIC for measurement in the full sample for each model
bic = [round(x.bic, 2) for x in [reg1, reg2, reg3, reg4]]
sg.add_line('BIC', bic, location=stargazer.LineLocation.FOOTER_BOTTOM)

# Display Stargazer table
sg

In [ ]:
# Plotting a bar chart to visualize the differences in BIC scores
bic_values = [reg1.bic, reg2.bic, reg3.bic, reg4.bic]

# Model names as labels
model_names = ['Model 1', 'Model 2', 'Model 3', 'Model 4']

# Customizing the bar chart
plt.figure(figsize=(6, 4))
plt.bar(model_names, bic_values, color='purple')

# Adjusting and giving a limited interval for the y axis to see the slight differences between the scores
plt.ylim(112000, 114000)
plt.title('BIC Values for Different Regression Models')
plt.xlabel('Regression Models')
plt.ylabel('BIC')
plt.show()

In [ ]:
# Plotting a bar chart to visualize the differences in RMSE scores
rmse_values = [rmse(reg1.fittedvalues,data.usd_price_day),
               rmse(reg2.fittedvalues,data.usd_price_day),
               rmse(reg3.fittedvalues,data.usd_price_day),
               rmse(reg4.fittedvalues,data.usd_price_day)]

# Model names as labels
model_names = ['Model 1', 'Model 2', 'Model 3', 'Model 4']

# Customizing the bar chart
plt.figure(figsize=(6, 4))
plt.bar(model_names, rmse_values, color='purple')

# Adjusting and giving a limited interval for the y axis to see the slight differences between the scores
plt.ylim(51, 59)
plt.title('RMSE Values for Different Regression Models')
plt.xlabel('Regression Models')
plt.ylabel('RMSE')
plt.show()

__________________________________________________________________
# 2. Model: LASSO
__________________________________________________________________

In [ ]:
# Defining variables

data['n_accommodates2'] = data['n_accommodates']**2
data['n_beds2'] = data['n_beds']**2
data['n_beds3'] = data['n_beds']**3

basic_lev = (
    "n_accommodates",
    "n_beds",
    "f_property_type",
    "f_room_type",
)
basic_add = ("f_bathrooms", "f_minimum_nights")
reviews = ("f_number_of_reviews", "n_review_scores_rating")
poly_lev = ("n_accommodates2", "n_beds2", "n_beds3")

#### Interactions

In [ ]:
#######################

#### Regressions with cross-validation

In [ ]:
# Defining different sets of features
X1 = ("f_room_type*f_property_type",  "f_room_type*d_host_is_superhost")
X2 = ("d_instant_bookable*f_property_type", "d_has_availability*f_property_type", "d_host_is_superhost*f_property_type")
X3 = ("(f_property_type + f_room_type + f_minimum_nights + n_beds)*d_host_is_superhost")

In [ ]:
# Creating 8 different models
modellev1="~ n_accommodates"
modellev2="~"+"+".join(basic_lev)
modellev3="~"+"+".join(basic_lev)+"+"+"+".join(basic_add)+"+"+"+".join(reviews)
modellev4="~"+"+".join(basic_lev)+"+"+"+".join(basic_add)+"+"+"+".join(reviews)+"+"+"+".join(poly_lev)
modellev5="~"+"+".join(basic_lev)+"+"+"+".join(basic_add)+"+"+"+".join(reviews)+"+"+"+".join(poly_lev)+"+"+"+".join(X1)
modellev6="~"+"+".join(basic_lev)+"+"+"+".join(basic_add)+"+"+"+".join(reviews)+"+"+"+".join(poly_lev)+"+"+"+".join(X1)+"+"+"+".join(X2)
modellev7="~"+"+".join(basic_lev)+"+"+"+".join(basic_add)+"+"+"+".join(reviews)+"+"+"+".join(poly_lev)+"+"+"+".join(X1)+"+"+"+".join(X2)
modellev8="~"+"+".join(basic_lev)+"+"+"+".join(basic_add)+"+"+"+".join(reviews)+"+"+"+".join(poly_lev)+"+"+"+".join(X1)+"+"+"+".join(X2)+"+"+X3

In [ ]:
# Calculating the sample size
smp_size = round(0.2 * data.shape[0])-1
smp_size

In [ ]:
# Set the seed to make results reproducable
np.random.seed(20240115)

In [ ]:
# Splitting the train and hold out set
data_work,data_holdout=train_test_split(data, test_size=smp_size)

In [ ]:
# Checking the shape of the sets
print(data_work.shape)
print(data_holdout.shape)

In [ ]:
# Setting the n folds to 5 
n_folds=5

In [ ]:
# Splitting the data into 4 categories, assuming that the data is randomly ordered
k = KFold(n_splits=4, shuffle=False, random_state=None)

In [ ]:
def cv_reg(formula, data, kfold, testdata, robustse=None):
    regression_list = []
    predicts_on_test = []
    rsquared = []
    rmse_list = []
    rmse_list_test = []

    # Calculating OLS for each fold
    
    for train_index, test_index in k.split(data):
        data_train, data_test = data.iloc[train_index, :], data.iloc[test_index, :]
        if robustse is None:
            model = smf.ols(formula, data=data_train).fit()
        else:
            model = smf.ols(formula, data=data_train).fit(cov_type=robustse)
        regression_list += [model]
        predicts_on_test += [model.predict(data_test)]
        rsquared += [model.rsquared]

        rmse_tr = pd.concat(
            [data_train["usd_price_day"], model.predict(data_train)],
            axis=1,
            keys=["usd_price_day", "predicted"],
        )
        rmse_tr = rmse_tr[~rmse_tr.isna().any(axis=1)]

        rmse_te = pd.concat(
            [data_test["usd_price_day"], model.predict(data_test)],
            axis=1,
            keys=["usd_price_day", "predicted"],
        )
        rmse_te = rmse_te[~rmse_te.isna().any(axis=1)]

        rmse_list += [rmse(rmse_tr["usd_price_day"], rmse_tr["predicted"], axis=0)]
        rmse_list_test += [rmse(rmse_te["usd_price_day"], rmse_te["predicted"], axis=0)]
    nvars = model.df_model

    # We return a dictionary with the 
    return {
        "regressions": regression_list,
        "test_predict": predicts_on_test,
        "r2": rsquared,
        "rmse": rmse_list,
        "rmse_test": rmse_list_test,
        "nvars": nvars,
    }

# Summarize method --> gives us a df
def summarize_cv(cvlist, stat="rmse"):
    result = pd.DataFrame(
        {"Model" + str(x + 1): cvlist[x][stat] for x in range(len(cvlist))}
    )
    result["Resample"] = ["Fold" + str(x + 1) for x in range(len(cvlist[0]["rmse"]))]
    result = result.set_index("Resample")
    result = pd.concat([result, pd.DataFrame(result.mean(), columns=["Average"]).T])
    return result

In [ ]:
# Performing cross-validated linear regressions with cov.type = HC0
cv_list = []
for i in [
    modellev1,
    modellev2,
    modellev3,
    modellev4,
    modellev5,
    modellev6,
    modellev7,
    modellev8,
]:
    cv_list += [cv_reg("usd_price_day" + i, data, k, "HC0")]

In [ ]:
# Summarizing the cross-validation results
summarize_cv(cv_list)

In [ ]:
# Creating a DataFrame for the RMSE values of the train and the test set
rmse_levels = {"nvars": [], "var": [], "value": []}
for i in range(0, 8):
    rmse_levels["nvars"].append(int(cv_list[i]["nvars"]))
    rmse_levels["var"].append("RMSE Training")
    rmse_levels["value"].append(pd.Series(cv_list[i]["rmse"]).mean())
for i in range(0, 8):
    rmse_levels["nvars"].append(int(cv_list[i]["nvars"]))
    rmse_levels["var"].append("RMSE Test")
    rmse_levels["value"].append(pd.Series(cv_list[i]["rmse_test"]).mean())
rmse_levels = pd.DataFrame.from_dict(rmse_levels)
rmse_levels["nvars2"] = rmse_levels["nvars"] + 1

In [ ]:
rmse_levels

In [ ]:
# Visualizing the RMSE values of the train and the test set
(
    ggplot(rmse_levels, aes(x="factor(nvars2)", y="value", color="var", group="var"))
    + geom_line(size=1, show_legend=True, na_rm=True)
    + scale_color_manual(name="", values=(color[0], color[1]))
    + scale_y_continuous(name="RMSE", limits=(50, 70), breaks=seq(50, 70, 2))
    + scale_x_discrete(
        name="Number of coefficients",
        expand=(0.01, 0.01)
    )
    + theme_bw()
)

The RMSE values of the train and the test set seem to align and go to the same direction, suggesting that the model is not overfitting and the model is generalizing well to unseen data. The RMSE values of the test set suggest that the model is improving in its predictive power, but there is a slight split when the number of coefficients reaches 18. Despite the good performance, further analysis and evaluation are needed to determine the best model for the specific context and requirements.

#### Doing LASSO in another way to check out lambda values

In [ ]:
# Defining the model evaluation method and splitting the dataset into 5 folds
cv = RepeatedKFold(n_splits=5, n_repeats=1, random_state=1)
y, X = patsy.dmatrices("usd_price_day" + modellev8, data)

In [ ]:
# Scaling the data
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Preparing for Ridge regression with cross-validation
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=smp_size, random_state=10
)

# Defining a range of lambda values
lambdas = [i/100 for i in range(5, 100,5)]

# Creating arrays to store R-squared values
train_r_squared = np.zeros(len(lambdas))
test_r_squared = np.zeros(len(lambdas))

pred_num = X.shape[1]
coeff_a = np.zeros((len(lambdas), pred_num))

In [ ]:
import datetime
from sklearn.model_selection import cross_val_score

for ind, i in enumerate(lambdas):
    print(f"Run: {str(ind).rjust(2)}, lambda: {i:.2f}, start: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    
    reg = Lasso(alpha = i, random_state= 20240211)
    reg.fit(X_train, y_train)
    results = cross_val_score(reg, X, y, cv=cv, scoring="r2")

    train_r_squared[ind] = reg.score(X_train, y_train)    
    test_r_squared[ind] = reg.score(X_test, y_test)

In [ ]:
# Melting the R-squared data into a DataFrame
r_squared_data = (
    pd.DataFrame(
        {
            "$R^2$ Test set": test_r_squared,
            "$R^2$ Training set": train_r_squared,
            "lambda": lambdas,
        }
    )
    .melt(id_vars=["lambda"])
)

In [ ]:
# Transforming the variables into categorical and reordering
r_squared_data["variable"] = r_squared_data["variable"].astype("category").cat.reorder_categories(
    ["$R^2$ Training set", "$R^2$ Test set"]
)

In [ ]:
r_squared_data

In [ ]:
# Plotting the R2 results
(
    ggplot(r_squared_data, aes(x="lambda", y="value", color="variable"))
    + geom_point()
    + geom_line(size=1, show_legend=False, na_rm=True)
    + scale_color_manual(name="", values=(color[1], color[0]))
    + scale_y_continuous(name="$R^2$")
    + facet_wrap("variable", scales="free")
    + theme_bw()
    + theme(subplots_adjust={"wspace": 0.25},legend_title=element_blank())
)

Separating the R2 values gave us another perspective on fit and model performance. Similar to the other LASSO results, the values indicate that the model is fitting well to both data sets. The R2 values of the training set are higher than the test set which was expected. The R-squared values of the test set are the highest when lambda equals to 0.05 and the values decrease gradually as lambda increases. This suggest thata lower lambda value might be more optimal to achieve a better fit on unseen data. However, the decreasing trend in the R2 values in the test set indicates that regularization (a higher lambda) helps prevent overfitting.

__________________________________________________________________
# 3. Model: Random Forest
__________________________________________________________________

In [ ]:
# Setting the train and hold out set
data_train, data_holdout = train_test_split( data, train_size=0.7, random_state=42)

In [ ]:
# Checking the shape of the train set and hold out set
data_train.shape, data_holdout.shape

In [ ]:
# Creating a list for basic variables
basic_vars = [
    "n_accommodates",
    "n_beds",
    "f_property_type",
    "f_room_type",
    "f_bathrooms",
    "f_minimum_nights",
    "n_maximum_nights",
    "f_neighbourhood_cleansed",
]

# Creating a list for reviews variables
reviews = [
    "f_number_of_reviews",
    "n_review_scores_rating",
    "n_reviews_per_month",
]

# Creating amenities from the dummy variables
amenities = [col for col in data if col.startswith("d_")]

In [ ]:
amenities

In [ ]:
# Creating interactions for LASSO
X1 = [
    "n_accommodates:f_property_type",
    "f_room_type:f_property_type",
    "f_room_type:d_has_availability",
    "f_bathrooms:f_property_type",
    "f_neighbourhood_cleansed:f_property_type",
    "d_instant_bookable:f_minimum_nights",
]
# with boroughs
X2 = [
    "f_property_type:f_neighbourhood_cleansed",
    "f_room_type:f_neighbourhood_cleansed",
    "n_accommodates:f_neighbourhood_cleansed",
]

In [ ]:
# Creating different predictors, including the basic variables, amenities and the interaction terms
predictors_1 = basic_vars
predictors_2 = basic_vars + reviews + amenities
predictors_E = basic_vars + reviews + amenities + X1 + X2

In [ ]:
# Creating the design matrices that represents the predictors_2 and the target variable in a structured format
y, X = dmatrices("usd_price_day ~ " + " + ".join(predictors_2), data_train)

In [ ]:
X

In [ ]:
X.design_info.column_names

In [ ]:
np.asarray(X)

In [ ]:
# According to the theory, the optimal number of trees is the square root of the number of variables we obtain
print('The theoretical recommended number of variables: {:.2f}.'.format(math.sqrt(len(X.design_info.column_names))))

In [ ]:
# Checking the shape of the X matrice
X.shape

In [ ]:
type(y)

In [ ]:
# Checking the shape of the y matrice
y.shape

In [ ]:
# Transforming y to one-dimensional to make it compatible for the modeling algorithm
y.ravel().shape

In [ ]:
# Creating a GridSearch
rfr = RandomForestRegressor(random_state = 20240211)

In [ ]:
# Searching for every potential parameter
tune_grid = {"max_features": [6, 8, 10, 12], "min_samples_leaf": [5, 10, 15]}

In [ ]:
# Using GridSearch to tune hyperparameters of an estimator and find the best combination of parameter values
rf_random = GridSearchCV(
    rfr,
    tune_grid,
    cv=5,
    scoring="neg_root_mean_squared_error",
    verbose=3,
)

In [ ]:
%%time

# Triggering the GridSearch process
# Evaluating different combinations of hyperparameters using CV
rf_model = rf_random.fit(X, y.ravel())

In [ ]:
# Getting the cross-valudates results
rf_model.cv_results_

In [ ]:
# Placing the cross-validated results into a DataFrame
df_rf_model_cv_results = pd.DataFrame(rf_model.cv_results_)[[
    'param_max_features', 'param_min_samples_leaf', 'mean_test_score']]

In [ ]:
# Renaming the columns
df_rf_model_cv_results.columns = ['Max features', 'Min node size', 'RMSE']

In [ ]:
# Checking the RMSE of the cv results
df_rf_model_cv_results

In [ ]:
# Reshaping the DataFrame and rounding the values
df_rf_model_cv_results.pivot(
    index = "Max features",
    columns = "Min node size",
    values = "RMSE").round(2)*-1

In [ ]:
# Filtering for the best score with scikit syntax
rf_model.best_score_

In [ ]:
# Filtering for the best parameters with scikit syntax
rf_model.best_params_

In [ ]:
# Concluding the best estimator
rf_model.best_estimator_

----------------------------------------------------------------------------------------------------------------
## Diagnostics

### Feature importances

In [ ]:
# Retrieving the feature importances calculated by the best estimator
rf_model.best_estimator_.feature_importances_

In [ ]:
# Creating a DataFrame to visualize the feature importances and their corresponding column names
pd.DataFrame(
    rf_model.best_estimator_.feature_importances_, 
    X.design_info.column_names)

In [ ]:
# Reshaping the DataFrame for better readability
# Resetting the index for proper alignment
# Renaming the columns to 'variable' and to 'imp'
# Sorting the feature importance values by descending order
# Resetting the index again and dropping the old ones
df_var_imp = pd.DataFrame(
    rf_model.best_estimator_.feature_importances_, 
    X.design_info.column_names)\
    .reset_index()\
    .rename({"index": "variable", 0: "imp"}, axis=1)\
    .sort_values(by=["imp"], ascending=False)\
    .reset_index(drop = True)

# Calculating the cumulative sum of feature importance values
df_var_imp['cumulative_imp'] = df_var_imp['imp'].cumsum()

In [ ]:
# Formatting the feature importance and cumulative importance columns as percentages for an easier interpreation
df_var_imp.style.format({
    'imp': lambda x: f'{x:,.1%}',
    'cumulative_imp': lambda x: f'{x:,.1%}'})

In [ ]:
# Setting the cutoff to 1%
cutoff = 0.01

In [ ]:
# Plotting feature importances with horizontal bar charts
df_var_imp[df_var_imp.imp > cutoff]\
    .sort_values(by = 'imp')\
    .plot(kind = 'barh', 
          x = 'variable', y = 'imp', 
          figsize = (10,6), grid = True, 
          title = 'Random forest model highest feature importances', 
          ylabel = 'Variables', legend = False
         );

#### Creating an OLS linear regression with the variables obtaining the highest feature importance values

In [ ]:
# Creating the regression
reg5 = smf.ols(formula="usd_price_day~f_room_type + n_accommodates + n_beds", data=data).fit(cov_type="HC1")
reg5.summary()

In [ ]:
# Comparing the first linear regression with reg5, the "optimal" regression model
sg = stargazer.Stargazer([reg1, reg5])

# Setting the covariate order
covariate_order = ["f_room_type[T.Private]", "f_room_type[T.Shared]",
                   "n_accommodates", "n_beds", "Intercept"]
sg.covariate_order(covariate_order)

sg.rename_covariates({"n_accommodates": "Accommodates", 
                      "f_room_type[T.Private]": "Private Room Type", 
                      "f_room_type[T.Shared]": "Shared Room Type", 
                      "n_beds" : "Beds",
                      "Intercept": "Constant"})

# Adding RMSE for measurement in the full sample for each model
rmse_values = [round(rmse(x.fittedvalues, data.usd_price_day), 3) for x in [reg1, reg5]]
sg.add_line('RMSE', rmse_values, location=stargazer.LineLocation.FOOTER_BOTTOM)

# Adding BIC for measurement in the full sample for each model
bic = [round(x.bic, 2) for x in [reg1, reg5]]
sg.add_line('BIC', bic, location=stargazer.LineLocation.FOOTER_BOTTOM)

sg

The BIC values for the RandomForest model (112540.63) are lower than those for the manually selected variables model (113928.12), indicating a better fit. Similarly, the RMSE values are lower for the RandomForest model (54.705) than the first regression (58.566), which suggests better predictive accuracy. Overall, using RandomForest to determine the most important variables was a good decision because the 5th regression model not only captures the patterns more effecitvely but provides better predictions for the dependent variable.